In [26]:
import torch
import numpy as np
torch.cuda.is_available()

True

In [27]:
# read shakeaspeare.txt

with open('sheksper.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [28]:
print('length of text: {} characters'.format(len(text)))

length of text: 1115394 characters


In [29]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [30]:
chars_in_text = sorted(list(set(text)))
c_to_i = {c:i for i,c in enumerate(chars_in_text)}
i_to_c = {i:c for i,c in enumerate(chars_in_text)}

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'   

In [32]:
vocab_size = len(chars_in_text)

In [33]:
vocab_size

65

In [34]:
print(''.join(chars_in_text))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [35]:
encode = lambda s: [c_to_i[c] for c in s] # for every char inside the string return the integer value
decode = lambda s: ''.join([i_to_c[i] for i in s]) # for every integer inside the string return the char value

encode('hello')

[46, 43, 50, 50, 53]

In [36]:
decode(encode('Frank'))

'Frank'

In [37]:
# using torch create an encoding of the text

encoded = torch.tensor(encode(text)).long()

In [38]:
encoded

tensor([18, 47, 56,  ..., 45,  8,  0])

In [39]:
encoded.shape

torch.Size([1115394])

In [40]:
encoded.dtype

torch.int64

In [41]:
# create a dataset with training and validation split

training_data = encoded[:int(len(encoded)*0.9)]
validation_data = encoded[int(len(encoded)*0.9):]

In [42]:
print(training_data[:1000])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [43]:
block_size = 8
x = training_data[:block_size]
y = training_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}")

When input is tensor([18]), the target is 47
When input is tensor([18, 47]), the target is 56
When input is tensor([18, 47, 56]), the target is 57
When input is tensor([18, 47, 56, 57]), the target is 58
When input is tensor([18, 47, 56, 57, 58]), the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [44]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] th

In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [46]:
torch.manual_seed(1337)

In [47]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embed_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets = None):
        logits = self.token_embed_table(idx) # (B,T,C
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            next_idx = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat([idx, next_idx], dim = 1)
        return idx
m = BigramModel(vocab_size)
m.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
logits


torch.Size([32, 65])
tensor(4.8786, device='cuda:0', grad_fn=<NllLossBackward0>)


tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [48]:
eval_iters = 200
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out
optimizer = torch.optim.AdamW(m.parameters(), lr=2e-2)

In [50]:
max_iters = 20_000
eval_interval = 300
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Step: 0, train loss: 4.730252265930176, val loss: 4.721394062042236
Step: 300, train loss: 2.7915990352630615, val loss: 2.8097448348999023
Step: 600, train loss: 2.5457162857055664, val loss: 2.574522018432617
Step: 900, train loss: 2.526674747467041, val loss: 2.551266670227051
Step: 1200, train loss: 2.5079894065856934, val loss: 2.518145799636841
Step: 1500, train loss: 2.480046272277832, val loss: 2.524364471435547
Step: 1800, train loss: 2.4635956287384033, val loss: 2.5036563873291016
Step: 2100, train loss: 2.481626033782959, val loss: 2.4966864585876465
Step: 2400, train loss: 2.498694896697998, val loss: 2.5025129318237305
Step: 2700, train loss: 2.4776291847229004, val loss: 2.499884605407715
Step: 3000, train loss: 2.493823766708374, val loss: 2.536231756210327
Step: 3300, train loss: 2.4486494064331055, val loss: 2.521982192993164
Step: 3600, train loss: 2.4879229068756104, val loss: 2.5337212085723877
Step: 3900, train loss: 2.4708707332611084, val loss: 2.499964952468872

In [57]:
context = torch.zeros((1,1), dtype=torch.long).to(device)
m.generate(context, 100)
print(decode(m.generate(context, 100)[0].tolist()))


UMe t jerak's: t wothr u th?
UShaing mbs, h n t il c; d jenlotir Whth'slld my ay meras be:
HERGSe nv


In [58]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [63]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        x_prev = x[b,:t+1]
        xbow[b,t] = torch.mean(x_prev, 0)

In [65]:
xbow[0], x[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]))

In [66]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [69]:
# use softmax to compute masked attention

tril = torch.tril(torch.ones(T,T))

In [70]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [71]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow3, xbow)

True

In [ ]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embed_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets = None):
        logits = self.token_embed_table(idx) # (B,T,C
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            next_idx = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat([idx, next_idx], dim = 1)
        return idx
m = BigramModel(vocab_size)
m.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
logits
